In [109]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
from datetime import datetime
import re

path_list = []
Tk().withdraw()
filename = askopenfilenames()
for path in filename:
    path_list.append(path)

df = pd.concat(
    map(pd.read_csv,path_list), ignore_index=True)

In [122]:
def mean_on_column(name_of_race, name_of_column,df1):
    rslt_df = df1.loc[df1['KRM'] == name_of_race,[name_of_column]].mean()[0]
    
    return rslt_df

def sum_on_column(name_of_race, name_of_column):
    rslt_df = df1.loc[df1['KRM'] == name_of_race,[name_of_column]].sum()[0]
    rslt_df = round(rslt_df, 2)
    
    return rslt_df

def hybryd_mean(name_of_column,df1):
    rslt_df = df1[name_of_column].mean()
    
    return rslt_df

def process(x):
    pattern = re.compile(r'[ ][\d]+')
    if isinstance(x, str):
        result = re.search(pattern, x)
        if result:
            x = result.group().strip()
    return x

In [123]:
#Wyznaczam Przedział czasowy z jakiego mają być brane dane
less_then_2021 = df['KDOC']<= '2020-12-31'
more_then_2020 = df['KDOC']>= '2020-01-01'
df = df[less_then_2021 & more_then_2020]

In [124]:
#Liczę wiek badanego prosiaka
df['KDOC'] = pd.to_datetime(df['KDOC'])
df['KDUR'] = pd.to_datetime(df['KDUR']) 
df['Wiek'] =  df['KDOC'] - df['KDUR'] 

#Tworzę nową kolumnę średniej grubości z wzoru
df['Średnia grubość słoniny standar. (mm)'] = (df.KY3 + df.KY5)/2

#Filtruję df tak żeby odrzucić mieszańce
hybryd1 = df['KRO']
hybryd2 = df['KRM']
nonHybryd = hybryd1 == hybryd2
hybryd = hybryd1!=hybryd2
df1 = df[nonHybryd]
df3 = df[hybryd]

#Wyodrębniam numery wszystkich ras zawierających się w df1
list_of_race= list(df1['KRM'].unique())

#deklaruję listę niezbędnych kolumn
list_of_column = ['KP','Wiek','KY1','Średnia grubość słoniny standar. (mm)','OKO','T_PRZYR','MIESO_PROC']

list_of_race

[50, 10, 20, 40, 70, 30, 60, 80, 90]

In [125]:
# funkcja wylicza średnie wyniki dla większości kolumn dla liczby knórów liczy ile ich jest
def create_final_df(df1):
    final_list = {}
    
    
    for i in list_of_column:
        temp_list = []
        
        for j in list_of_race:
            if i == 'KP':
                #Liczę liczbę knurów ze względu na rasę
                temp_series = df1['KRM'].where(df1['KP'] == 1).dropna().value_counts()
                temp_list.append(temp_series[j])
            
            #Liczę średnie wyniki pozostałych kolumn
            else:
                if i == "KY1":
                    temp_list.append(round(mean_on_column(j,i,df1),0))
                elif i != "Wiek":
                    temp_list.append(round(mean_on_column(j,i,df1),2))
                else:
                    temp_list.append(mean_on_column(j,i,df1))
                
            #słownik zewnętrzny przyjmuje wyliczone wartości 
            final_list[i] = temp_list
    final_list
    #Przetworzenie słownika na DF
    return pd.DataFrame.from_dict(final_list, orient="index", columns=list_of_race).T

In [145]:
# funkcja bliźniacza licząca średnie wyniki dla mieszańców
def hybrid_dic(df3):
    final_list = {}
    
    for i in list_of_column:
        temp_list = []

        if i == 'KP':
            temp_series = df3['KP'].where(df3['KP'] == 1).dropna().value_counts()
            temp_list.append(temp_series)

        else:
            if i == "KY1":
                temp_list.append(round(hybryd_mean(i,df1),0))
            elif i != "Wiek":
                temp_list.append(round(hybryd_mean(i,df1),2))
            else:
                temp_list.append(hybryd_mean(i,df1),)
        final_list[i] = temp_list
    return final_list

In [146]:
#Tworzę df z wynikami mieszańców
hybrid_dic = hybrid_dic(df3)
df0= pd.DataFrame(hybrid_dic)

In [147]:
#Tworzę finalną tabelę i dodaję do niej mieszańców
df2 = create_final_df(df1)
df2 = df2.append(df0,ignore_index=True)

#Obcinam zbędny tekst z kolumny "Wiek"
df2['Wiek'] = df2['Wiek'].astype(str)
df2["Wiek"] = df2["Wiek"].str.slice(start=0,stop=3)

In [148]:

df2

,KP,Wiek,KY1,Średnia grubość słoniny standar. (mm),OKO,T_PRZYR,MIESO_PROC
0,12,204,88.0,19.61,40.88,424.26,47.97
1,526,172,112.0,10.0,59.25,673.57,59.69
2,1314,173,116.0,9.84,58.34,690.41,59.66
3,660,185,106.0,12.71,53.15,566.47,56.21
4,208,173,121.0,9.56,61.61,717.94,61.06
5,15,196,101.0,18.0,46.08,508.71,50.68
6,15,165,132.0,8.01,66.35,837.6,63.37
7,196,186,124.0,7.89,65.77,661.47,63.8
8,116,168,106.0,9.58,58.6,655.73,59.44
9,"1.0 1393 Name: KP, dtype: int64",176,113.0,10.59,57.51,654.31,58.9


In [121]:
#Nadaję odpowiednie nazwy kolumną jak i indexom 
df2.rename(columns={
    'KP': 'Liczba knurów',
    'KY1': 'Masa ciała w dniu oceny (kg)',
    'OKO': 'Wysokość "oka" polędwycy standar. (mm)',
    'T_PRZYR': 'Przyrost dznienny standar. (g)', 'MIESO_PROC': 'Procentowa zawartość mięsa standar.'},
    index={10: 'WBP',20: 'PBZ',30: 'Złotnicka Biała', 40: 'Puławska',50: 'Złotnicka Pstra', 70: 'Duroc', 60: 'Hampshire', 80: 'Pietrain',90: 'Linia 990'},inplace = True)

In [142]:
df2

,KP,Wiek,KY1,Średnia grubość słoniny standar. (mm),OKO,T_PRZYR,MIESO_PROC
0,NaN,204,88.0,19.61,40.88,424.26,47.97
1,NaN,172,112.0,10.0,59.25,673.57,59.69
2,NaN,173,116.0,9.84,58.34,690.41,59.66
3,NaN,185,106.0,12.71,53.15,566.47,56.21
4,NaN,173,121.0,9.56,61.61,717.94,61.06
5,NaN,196,101.0,18.0,46.08,508.71,50.68
6,NaN,165,132.0,8.01,66.35,837.6,63.37
7,NaN,186,124.0,7.89,65.77,661.47,63.8
8,NaN,168,106.0,9.58,58.6,655.73,59.44
9,NaN,176,113.0,10.59,57.51,654.31,58.9
